# Proyecto de pruebas A/B

Has recibido una tarea analítica de una tienda en línea internacional. Tus predecesores no consiguieron completarla: lanzaron una prueba A/B y luego abandonaron (para iniciar una granja de sandías en Brasil). Solo dejaron las especificaciones técnicas y los resultados de las pruebas.

Descripción técnica
- Nombre de la prueba: recommender_system_test
- Grupos: А (control), B (nuevo embudo de pago)
- Fecha de lanzamiento: 2020-12-07
- Fecha en la que dejaron de aceptar nuevos usuarios: 2020-12-21
- Fecha de finalización: 2021-01-01
- Audiencia: 15% de los nuevos usuarios de la región de la UE
- Propósito de la prueba: probar cambios relacionados con la introducción de un sistema de recomendaciones mejorado
- Resultado esperado: dentro de los 14 días posteriores a la inscripción, los usuarios mostrarán una mejor conversión en vistas de la página del producto (el evento product_page), instancias de agregar artículos al carrito de compras (product_cart) y compras (purchase). En cada etapa del embudo product_page → product_cart → purchase, habrá al menos un 10% de aumento.
- Número previsto de participantes de la prueba: 6 000

## Paso 1. Importación de librerías y carga de dataset.

In [1]:
# Importar librerías

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import norm
import numpy as np

In [2]:
# Cargar los datasets

marketing_events = pd.read_csv('/datasets/ab_project_marketing_events_us.csv')
new_users = pd.read_csv('/datasets/final_ab_new_users_upd_us.csv')
events = pd.read_csv('/datasets/final_ab_events_upd_us.csv')
participants = pd.read_csv('/datasets/final_ab_participants_upd_us.csv')

## Paso 2. Llevar a cabo el análisis exploratorio de datos (EDA)

- ¿Es necesario convertir los tipos?
- ¿Hay valores ausentes o duplicados? Si es así, ¿cómo los caracterizarías?
- Estudia la conversión en las diferentes etapas del embudo.
- ¿El número de eventos por usuario está distribuido equitativamente entre las muestras?
- ¿Hay usuarios que están presentes en ambas muestras?
- ¿Cómo se distribuye el número de eventos entre los días?
- ¿Hay alguna peculiaridad en los datos que hay que tener en cuenta antes de iniciar la prueba A/B?

### ¿Es necesario convertir los tipos?

In [3]:
marketing_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


In [4]:
new_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     58703 non-null  object
 1   first_date  58703 non-null  object
 2   region      58703 non-null  object
 3   device      58703 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [5]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     423761 non-null  object 
 1   event_dt    423761 non-null  object 
 2   event_name  423761 non-null  object 
 3   details     60314 non-null   float64
dtypes: float64(1), object(3)
memory usage: 12.9+ MB


In [6]:
participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB


In [7]:
# Convertir las columnas de fechas a formato de fecha

marketing_events['start_dt'] = pd.to_datetime(marketing_events['start_dt'])
marketing_events['finish_dt'] = pd.to_datetime(marketing_events['finish_dt'])
new_users['first_date'] = pd.to_datetime(new_users['first_date'])
events['event_dt'] = pd.to_datetime(events['event_dt'])

# Convertir la columna 'details' a numérico (float), manejar NaN como 0 para análisis

events['details'] = pd.to_numeric(events['details'], errors='coerce').fillna(0)

marketing_events_info_after = marketing_events.info()
new_users_info_after = new_users.info()
events_info_after = events.info()

marketing_events_info_after, new_users_info_after, events_info_after

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   name       14 non-null     object        
 1   regions    14 non-null     object        
 2   start_dt   14 non-null     datetime64[ns]
 3   finish_dt  14 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 576.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     58703 non-null  object        
 1   first_date  58703 non-null  datetime64[ns]
 2   region      58703 non-null  object        
 3   device      58703 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total

(None, None, None)

- Las fechas en los conjuntos de datos de eventos de marketing, nuevos usuarios y eventos de usuarios ahora están en formato datetime.
- Los detalles de los eventos (details) se han convertido a formato numérico, con los valores ausentes manejados como 0.

### ¿Hay valores ausentes o duplicados? Si es así, ¿cómo los caracterizarías?

In [8]:
# Verificar valores ausentes

marketing_events_missing = marketing_events.isnull().sum()
new_users_missing = new_users.isnull().sum()
events_missing = events.isnull().sum()
participants_missing = participants.isnull().sum()

display(marketing_events_missing, new_users_missing, events_missing, participants_missing)

name         0
regions      0
start_dt     0
finish_dt    0
dtype: int64

user_id       0
first_date    0
region        0
device        0
dtype: int64

user_id       0
event_dt      0
event_name    0
details       0
dtype: int64

user_id    0
group      0
ab_test    0
dtype: int64

- Hay 363,447 valores nulos en la columna details del dataset 'final_ab_events_upd_us.csv'. Esto es comprensible porque la columna details solo contiene información adicional, como el monto de la compra, que no se aplica a todos los eventos.

In [9]:
# Verificar duplicados

marketing_events_duplicates = marketing_events.duplicated().sum()
new_users_duplicates = new_users.duplicated().sum()
events_duplicates = events.duplicated().sum()
participants_duplicates = participants.duplicated().sum()

display(marketing_events_duplicates, new_users_duplicates, events_duplicates, participants_duplicates)

0

0

0

0

- No se encontraron duplicados en ninguno de los conjuntos de datos.

### Estudia la conversión en las diferentes etapas del embudo.

In [19]:
# Filtrar eventos por cada etapa del embudo

product_page_events = events[events['event_name'] == 'product_page']
product_cart_events = events[events['event_name'] == 'product_cart']
purchase_events = events[events['event_name'] == 'purchase']

# Contar eventos únicos por usuario en cada etapa

users_in_product_page = product_page_events['user_id'].nunique()
users_in_product_cart = product_cart_events['user_id'].nunique()
users_in_purchase = purchase_events['user_id'].nunique()

# Crear el gráfico tipo embudo

fig = go.Figure(go.Funnel(
    y = ["Product Page Views", "Added to Cart", "Purchases"],
    x = [users_in_product_page, users_in_product_cart, users_in_purchase],
    textinfo = "value+percent initial"
))

fig.update_layout(title="Conversion Funnel of Recommender System Test")
fig.show()

In [11]:
# Calcular las conversiones entre las etapas del embudo

conversion_to_cart = (users_in_product_cart / users_in_product_page) * 100 if users_in_product_page > 0 else 0
conversion_to_purchase = (users_in_purchase / users_in_product_cart) * 100 if users_in_product_cart > 0 else 0

# Resultados de la conversión

display(users_in_product_page,
users_in_product_cart,
users_in_purchase,
conversion_to_cart,
conversion_to_purchase)

38929

19284

19568

49.53633537979398

101.47272350134826

- Usuarios que vieron la página del producto: 38,929
- Usuarios que añadieron productos al carrito: 19,284
- Usuarios que realizaron compras: 19,568


- Conversión a añadir al carrito: 49.54%
- Conversión a compra desde el carrito: 101.50%

Observamos que hay una conversión inusualmente alta del carrito de compras a la compra (más del 100%), lo cual indica que podría haber inconsistencias en los datos o duplicaciones de eventos de compra.

### ¿El número de eventos por usuario está distribuido equitativamente entre las muestras?

In [12]:
# Calcular el número de eventos por usuario

events_per_user = events.groupby('user_id')['event_name'].count().reset_index()
events_per_user.columns = ['user_id', 'event_count']

# Distribuir eventos por grupo A y B

participants_with_events = participants.merge(events_per_user, on='user_id', how='left').fillna(0)
group_a_events = participants_with_events[participants_with_events['group'] == 'A']['event_count']
group_b_events = participants_with_events[participants_with_events['group'] == 'B']['event_count']

group_a_summary = group_a_events.describe()
group_b_summary = group_b_events.describe()

group_a_summary, group_b_summary

(count    8214.000000
 mean        7.146944
 std         4.041994
 min         1.000000
 25%         4.000000
 50%         6.000000
 75%         9.000000
 max        28.000000
 Name: event_count, dtype: float64,
 count    6311.000000
 mean        6.993028
 std         4.079862
 min         1.000000
 25%         4.000000
 50%         6.000000
 75%         9.000000
 max        32.000000
 Name: event_count, dtype: float64)

Grupo A:

- Número de usuarios: 8,214
- Promedio de eventos por usuario: 7.15
- Desviación estándar: 4.04
- Rango de eventos por usuario: de 1 a 28

Grupo B:

- Número de usuarios: 6,311
- Promedio de eventos por usuario: 6.99
- Desviación estándar: 4.08
- Rango de eventos por usuario: de 1 a 32

Los promedios de eventos por usuario son similares entre los grupos A y B, lo que nos dice que los eventos están distribuidos de forma bastante equitativa entre ambos grupos. La diferencia en los promedios no parece significativa.

### ¿Hay usuarios que están presentes en ambas muestras?

In [13]:
# Identificar usuarios presentes en ambos grupos A y B

duplicate_users = participants.groupby('user_id')['group'].nunique().reset_index()
duplicate_users_in_both_groups = duplicate_users[duplicate_users['group'] > 1]

# Contar cuántos usuarios están presentes en ambos grupos

num_users_in_both_groups = len(duplicate_users_in_both_groups)

print(num_users_in_both_groups)

441


- Hay 441 usuarios que están presentes en ambos grupos (A y B), lo cual puede influir en los resultados de la prueba A/B debido a la falta de independencia entre los grupos.

- La presencia de usuarios en ambos grupos podría sesgar los resultados. Sería recomendable excluir estos usuarios de la comparación para mantener la integridad del análisis A/B.

### ¿Cómo se distribuye el número de eventos entre los días?

In [14]:
# Agregar columna con la fecha del evento para agrupar por día

events['event_date'] = events['event_dt'].dt.date

# Calcular el número de eventos por día

events_per_day = events.groupby('event_date').size().reset_index(name='event_count')

# Mostrar la distribución del número de eventos por día

display(events_per_day.head(10), events_per_day.describe())

,event_date,event_count
0,2020-12-07,11385
1,2020-12-08,12547
2,2020-12-09,12122
3,2020-12-10,14077
4,2020-12-11,13864
5,2020-12-12,17634
6,2020-12-13,20985
7,2020-12-14,26184
8,2020-12-15,23469
9,2020-12-16,20909


,event_count
count,23.000000
mean,18424.391304
std,7651.968438
min,89.000000
25%,12483.500000
50%,19399.000000
75%,23871.000000
max,32559.000000


In [15]:
# Crear un gráfico de líneas para mostrar la distribución de eventos por día

fig = px.line(events_per_day, x='event_date', y='event_count', title='Distribución de Eventos por Día', 
              labels={'event_date': 'Fecha', 'event_count': 'Número de Eventos'})

fig.update_layout(xaxis_title='Fecha', yaxis_title='Número de Eventos')
fig.show()

El número de eventos por día varía significativamente:

- Media: 18,424 eventos por día.
- Desviación estándar: 7,652 eventos.
- Mínimo: 89 eventos (posible outlier).
- Máximo: 32,559 eventos.

La gran variabilidad en el número de eventos por día y el número tan bajo de eventos en algunos días nos dicen que podría haber irregularidades o eventos específicos que afectaron la actividad de los usuarios.

### ¿Hay alguna peculiaridad en los datos que hay que tener en cuenta antes de iniciar la prueba A/B?

- Hay una conversión inusualmente alta (>100%) del carrito a la compra, lo que indica posibles inconsistencias o eventos duplicados.
- Los eventos están distribuidos equitativamente entre los grupos A y B.
- 441 usuarios están presentes en ambos grupos A y B, lo que puede sesgar los resultados de la prueba.
- Variabilidad significativa en la cantidad de eventos por día, lo que podría indicar efectos de marketing u otros factores.

## Paso 3. Evaluar los resultados de la prueba A/B:

- ¿Qué puedes decir sobre los resultados de la prueba A/B?
- Utiliza una prueba z para comprobar la diferencia estadística entre las proporciones.

In [16]:
# Excluir usuarios duplicados en ambos grupos

clean_participants = participants[~participants['user_id'].isin(duplicate_users_in_both_groups['user_id'])]

# Contar usuarios únicos en cada grupo después de eliminar duplicados

group_a_clean = clean_participants[clean_participants['group'] == 'A']
group_b_clean = clean_participants[clean_participants['group'] == 'B']

# Calcular conversiones para cada grupo

group_a_conversions = events[events['user_id'].isin(group_a_clean['user_id']) & (events['event_name'] == 'purchase')]['user_id'].nunique()
group_b_conversions = events[events['user_id'].isin(group_b_clean['user_id']) & (events['event_name'] == 'purchase')]['user_id'].nunique()

# Total de usuarios en cada grupo

total_users_a = len(group_a_clean)
total_users_b = len(group_b_clean)

print(total_users_a)
print(total_users_b)

7773
5870


In [17]:
# Proporciones de conversión

p1 = group_a_conversions / total_users_a
p2 = group_b_conversions / total_users_b

# Proporción combinada

p_combined = (group_a_conversions + group_b_conversions) / (total_users_a + total_users_b)

p_combined

0.3251484277651543

In [18]:
# Calcular el estadístico Z

z_score = (p1 - p2) / np.sqrt(p_combined * (1 - p_combined) * (1/total_users_a + 1/total_users_b))

# Calcular el valor p

p_value = 1 - norm.cdf(abs(z_score))

z_score, p_value, p1, p2


(1.0196244768613194,
 0.15395329596201335,
 0.32870191689180495,
 0.320442930153322)

Se realizó una prueba z para comparar la diferencia estadística entre las proporciones de conversión de los grupos A y B, excluyendo a los usuarios presentes en ambas muestras:

- Conversión del grupo A (limpio): 32.87%
- Conversión del grupo B (limpio): 32.04%
- Estadístico Z: 1.02
- Valor p: 0.154

No se encontró una diferencia estadísticamente significativa entre los grupos A y B en términos de conversiones de compra (valor p > 0.05).

## Describe tus conclusiones con respecto a la etapa EDA y los resultados de la prueba A/B.

- La conversión desde la visualización de la página del producto hasta la compra muestra una tasa muy alta y fuera de lo regular (101.47%) al pasar del carrito a la compra. Este comportamiento sugiere la posibilidad de duplicados o inconsistencias en los datos de compra. Podría deberse a múltiples registros de eventos de compra por usuario o eventos incorrectamente etiquetados.

- La distribución de eventos por usuario entre los grupos A y B es bastante equitativa, con promedios similares (7.15 eventos en A y 6.99 en B). Esto indica que no hay un sesgo significativo en el número de interacciones de los usuarios en cada grupo.

- Se identificaron 441 usuarios presentes en ambos grupos (A y B). Este overlapping puede sesgar los resultados de la prueba A/B. Estos usuarios se excluyeron del análisis final para asegurar la validez de los resultados.

- Hay una variabilidad significativa en la cantidad de eventos por día, con un mínimo de 89 y un máximo de 32,559. Este rango amplio sugiere que puede haber días con actividad atípica, posiblemente debido a factores externos.

- Dado que el valor p es mayor que 0.05, no se puede rechazar la hipótesis nula, lo que indica que no hay una diferencia estadísticamente significativa en las conversiones entre los grupos A y B.